In [1]:
import re
import pandas as pd
import numpy as np
import json
import subprocess
import glob
import os

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors
from matplotlib import cm
matplotlib.style.use('ggplot')
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch

In [2]:
Mode = 'OneTest'

In [ ]:
Mode = 'Runs'

In [4]:
if Mode=='OneTest':
    eff_beg = 1.0
    eff_end = 1.0
    cost_beg = 1.0
    cost_end = 1.0
elif Mode == 'Runs':
    eff_beg = 1.0
    eff_end = 1.2
    cost_beg = 0.8
    cost_end = 1.2
    

In [5]:
N_SEEDS = 10
N_SEI = 20
N_TESTS = 1
N_TICKS = 260
N_TICKS_DISPLAY = 260 - 52
N_TICKS_BURN = N_TICKS - N_TICKS_DISPLAY

In [6]:
#read data into dataframe 
#append columns with eff and costs

def collect_data_for_simulation_runs_from_inst_eff(N_TESTS, dfs_inst, dfs_inst_std):

    test_mode = 'BaseVer02'
    data_mode = 'CA'
    #here think how to change it
    params_versions = [['eff' + str(x) + 'cost' + str(y) for y in np.linspace(cost_beg,cost_end,N_TESTS,endpoint=True)]\
                        for x in np.linspace(eff_beg,eff_end,N_TESTS,endpoint=True)]
    effs = [x for x in np.linspace(eff_beg,eff_end,N_TESTS,endpoint=True)]
    costs = [x for x in np.linspace(cost_beg,cost_end,N_TESTS,endpoint=True)]

    #names of columns in a file
    names = ['Tick', 'Location_x', 'Location_y', 'Uid', 'Price_per_watt', 'DC_size', 'Origin', 'Mart_type']


    for i_run in range(N_TESTS): 
        for j_run in range(N_TESTS):
            #read few files
            #save into database
            dfs_ = []
            #read last simulation result
            files = glob.glob('..\\..\\ABMIRISLab\\Saves\\May-14-2018\\BaseModel\\'+ test_mode + '\\' + data_mode + \
                     '\\' + params_versions[i_run][j_run] + '\\*[_pr].csv')
            
            files.sort(key=os.path.getmtime, reverse=True)

            #take top x files 
            for file in files[0 * N_SEEDS : 1 * N_SEEDS]:
                print(file)
                df_ = pd.read_csv(file, names=names, low_memory=False, na_values = ['-nan(ind)'])
                df_['eff'] = effs[i_run]
                df_['cost'] = costs[j_run]
                dfs_.append(df_)


            #create df from them    
            df_concat = pd.concat(dfs_)


            df_inst_mean = df_concat.groupby(level=0).mean()
            df_inst_std = df_concat.groupby(level=0).std()

            dfs_inst.append(dfs_)


In [15]:
dfs_inst_eff = []
dfs_inst_eff_std = []
collect_data_for_simulation_runs_from_inst_eff(1, dfs_inst_eff, dfs_inst_eff_std)

..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\b1d4a07b-a0e7-4505-b1c5-4753d1aff09b_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\0b35689d-6d37-4f75-83ea-009fc0584c8a_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\06b27367-d1bb-4a40-b03f-d77071ba5bdc_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\9772ed80-2a48-4926-a80c-e21bf8ad81af_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\e4624440-2854-45c5-b775-30ff5a4a003b_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\ddf52011-129b-449d-a29f-ccd929a56cb2_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\6486ce8f-8439-449f-8246-af09ff779cbd_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\ecf08955-1971-4708-9eae-001037bd182f_pr.csv
..\..\ABMIRISLab\Saves\May-14-2018\BaseModel\BaseVer02\CA\eff1.0cost1.0\

In [16]:
test = [[np.min(y[0]['eff']), np.min(y[0]['cost']), np.mean([len(y[i]) for i in range(len(y))])] for y in dfs_inst_eff]

In [17]:
data_heatmap = pd.DataFrame(test)

In [18]:
data_heatmap.columns= ['eff', 'cost', 'inst']

In [19]:
data_heatmap['inst_share'] = data_heatmap['inst']/10000

In [20]:
print(data_heatmap)

   eff  cost    inst  inst_share
0  1.0   1.0  2127.2     0.21272


In [ ]:
   eff  cost    inst  inst_share
0  1.0   1.0  1851.7     0.18517